# Model testing

- It's just a testing ground for the model. One model is loaded and a `cmc.test_model` is applied on it.
- As a result confusion matrix, ROC, AUROC, F1, PR curves will be generated and saved to the model's folder

In [1]:
import os
from pathlib import Path
import confinement_mode_classifier as cmc
import torchvision
import torch
from tqdm.auto import tqdm
import torch.nn as nn
import re

path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Seed set to 42


Device: cpu


In [4]:

data_dir_path = f'{path}/data/LH_alpha'
file_names = os.listdir(data_dir_path)


#Chose what shots will be used in notebook. Removed shots has different dimensions
batch_size = 32
shot_numbers = [re.search(r'shot_(\d+)', file_name).group(1) for file_name in file_names]
removed_shots = ['19915', '19925', '13182', '20009', '20112', 
                 '20143', '20145', '20146', '20147', '16989', 
                 '16987', '20144', '18263', '18267', '18266', 
                 '18279', '20098', '18260', '18200', '18268', '18261']
shot_numbers = [valid_shot for valid_shot in shot_numbers if valid_shot not in removed_shots]

shots_for_testing = ['18130', '16773', '16534', '19094', '18133', '17837', '18128']
shots_for_validation = ['16769', '19379', '18057', '18132']

shot_df, test_df, _, _ = cmc.load_and_split_dataframes(path, shot_numbers, shots_for_testing, 
                                                       shots_for_validation=shots_for_validation,
                                                       use_ELMS=False)


### Load the model

In [5]:
loader = cmc.get_dloader(shot_df, path=path, batch_size=batch_size, shuffle=False, 
                                   balance_data=False, ris_option='RIS1', second_img_opt='RIS2')


for batch in tqdm(loader):
    pass

  0%|          | 0/1832 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid 680239) is killed by signal: Killed. 

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 737, in start
    self.io_loop.start()
  File "/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 182

: 

In [ ]:
# #Initiate a single camera model
# pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = pretrained_model.fc.in_features
# pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
# pretrained_model = pretrained_model.to(device)

# #Which model to test?
# model_path = f'{path}/runs/17-01-2024, 13-25-47 RIS1xRIS2 no elms_all_layers'

# #TEST one camera model
# # pretrained_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
# # pretrained_model.eval()
# # pretrained_model.to(device)

# #TEST ensembled model
# ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
# ensembled_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
# ensembled_model.eval()
# ensembled_model.to(device)

# test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size, shuffle=False, 
#                                   balance_data=False, ris_option='RIS1', second_img_opt='RIS2')

### Test model on unbalanced test dset

In [8]:
metrics = cmc.test_model(model_path, ensembled_model, test_dataloader, max_batch=40)

Processing batches:   0%|          | 0/458 [00:00<?, ?it/s]

/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### Here the model is tested on individual shots (generates time-confidence graph)

In [ ]:
shots_for_testing = list(map(int,shots_for_testing))
img_path = cmc.per_shot_test(path=model_path, shots=shots_for_testing, results_df=metrics['prediction_df'])

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:

runs_dir_path = f'{path}/runs/'
runs_names = os.listdir(runs_dir_path)

In [19]:
runs_names.remove('07-02-24, 10-04-54 extended_dset_all_layers')

In [20]:


runs_dir_path = f'{path}/runs/'
runs_names = os.listdir(runs_dir_path)

no_elms_runs = [name for name in runs_names if 'elms on' not in name]
for subdir in tqdm(no_elms_runs):

    #Initiate a single camera model
    pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
    # Parameters of newly constructed modules have requires_grad=True by default
    num_ftrs = pretrained_model.fc.in_features
    pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
    pretrained_model = pretrained_model.to(device)

    run_path = f'{path}/runs/{subdir}'  # Construct the full path to the model

    print(subdir)


    # Otherwise, it's an ensembled model
    if len(re.findall(r'RIS[12]x', run_path)):
        # Extract the RIS names
        ris_names = re.findall(r'RIS[12]xRIS[12]', subdir)[0].split('x')
        first_cam, second_cam = ris_names[0], ris_names[1]
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
        ensembled_model.load_state_dict(torch.load(run_path+'/model.pt'))
        ensembled_model.eval()
        ensembled_model.to(device)

        test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size, shuffle=False, 
                                    balance_data=False, ris_option=first_cam, second_img_opt=second_cam)
        
        print(f' {first_cam} and {second_cam}')

        metrics = cmc.test_model(run_path, ensembled_model, test_dataloader)

        shots_for_testing = list(map(int,shots_for_testing))

        img_path = cmc.per_shot_test(path=run_path, shots=shots_for_testing, results_df=metrics['prediction_df'])

    # Check if it's a simple model
    elif len(re.findall(r'RIS[12]', run_path)):

        cam = re.findall(r'RIS[12]', run_path)

        pretrained_model.load_state_dict(torch.load(run_path+'/model.pt'))
        pretrained_model.eval()
        pretrained_model.to(device)

        test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size, shuffle=False, 
                                    balance_data=False, ris_option=cam)
        
        metrics = cmc.test_model(run_path, pretrained_model, test_dataloader)
        shots_for_testing = list(map(int,shots_for_testing))
        img_path = cmc.per_shot_test(path=run_path, shots=shots_for_testing, results_df=metrics['prediction_df'])
        # Optionally, do something with first_cam and second_cam
        
    else:
        pass

  0%|          | 0/17 [00:00<?, ?it/s]

18-01-24, 20-11-24 RIS2 no elms_all_layers


/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Processing batches:   0%|          | 0/436 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

Processing batches:   0%|          | 0/436 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

06-02-24, 18-34-57 extended_dset_last_fc
18-01-24, 20-17-11 RIS1xRIS1 no elms_classifier_training
 RIS1 and RIS1


/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Processing batches:   0%|          | 0/436 [00:00<?, ?it/s]